<a href="https://colab.research.google.com/github/connorpheraty/DS-Unit-1-Sprint-2-Data-Wrangling/blob/master/Connor_Heraty_DS_Unit_1_Sprint_Challenge_2_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 2

## Data Wrangling

In this Sprint Challenge you will use data from [Gapminder](https://www.gapminder.org/about-gapminder/), a Swedish non-profit co-founded by Hans Rosling. "Gapminder produces free teaching resources making the world understandable based on reliable statistics."
- [Cell phones (total), by country and year](https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--cell_phones_total--by--geo--time.csv)
- [Population (total), by country and year](https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--population_total--by--geo--time.csv)
- [Geo country codes](https://github.com/open-numbers/ddf--gapminder--systema_globalis/blob/master/ddf--entities--geo--country.csv)

These two links have everything you need to successfully complete the Sprint Challenge!
- [Pandas documentation: Working with Text Data](https://pandas.pydata.org/pandas-docs/stable/text.html]) (one question)
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf) (everything else)

## Part 0. Load data

You don't need to add or change anything here. Just run this cell and it loads the data for you, into three dataframes.

In [0]:
import pandas as pd

cell_phones = pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--cell_phones_total--by--geo--time.csv')

population = pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--population_total--by--geo--time.csv')

geo_country_codes = (pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--entities--geo--country.csv')
                       .rename(columns={'country': 'geo', 'name': 'country'}))

## Part 1. Join data

First, join the `cell_phones` and `population` dataframes (with an inner join on `geo` and `time`).

The resulting dataframe's shape should be: (8590, 4)

In [345]:
cell_phones.head(1)

,geo,time,cell_phones_total
0,abw,1960,0.0


In [0]:
cell_phones['cell_phones_total'] = cell_phones['cell_phones_total']

In [347]:
cell_phones.head(1)

,geo,time,cell_phones_total
0,abw,1960,0.0


In [348]:
population.head(1)

,geo,time,population_total
0,afg,1800,3280000


In [349]:
geo_country_codes.head(1)

,geo,alt_5,alternative_1,alternative_2,alternative_3,alternative_4_cdiac,arb1,arb2,arb3,arb4,...,latitude,longitude,main_religion_2008,country,pandg,un_state,unicode_region_subtag,upper_case_name,world_4region,world_6region
0,abkh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,nan,NaN,Abkhazia,NaN,False,NaN,NaN,europe,europe_central_asia


In [0]:
cell_df = pd.merge(cell_phones, population, on=['geo','time'])

In [351]:
cell_df.head()

,geo,time,cell_phones_total,population_total
0,afg,1960,0.0,8996351
1,afg,1965,0.0,9938414
2,afg,1970,0.0,11126123
3,afg,1975,0.0,12590286
4,afg,1976,0.0,12840299


In [352]:
cell_df.shape

(8590, 4)

Then, select the `geo` and `country` columns from the `geo_country_codes` dataframe, and join with your population and cell phone data.

The resulting dataframe's shape should be: (8590, 5)

I included the **`world_4region`** and **`world_6region`** to see the relationship between continent/subcontinent and cell phone adoption.

In [0]:
cell_df = cell_df.merge(geo_country_codes[['geo','country','world_4region','world_6region']])

In [0]:
cell_df = cell_df.rename({'world_4region':'region','world_6region':'subregion'},axis=1)

In [355]:
# Shape is expected with reion and subregion features added
cell_df.shape

(8590, 7)

## Part 2. Make features

Calculate the number of cell phones per person, and add this column onto your dataframe.

(You've calculated correctly if you get 1.220 cell phones per person in the United States in 2017.)

In [0]:
# Create cell_phones_per_person column using column division
cell_df['cell_phones_per_person'] = cell_df['cell_phones_total'] / cell_df['population_total']

In [0]:
# Round the calculation to 4 decimal places
cell_df['cell_phones_per_person'] = cell_df['cell_phones_per_person'].round(decimals=4)

In [358]:
# Locate row containing 2017 United States data
cell_df.loc[cell_df['country'] == 'United States'][::-1].head(1)

,geo,time,cell_phones_total,population_total,country,region,subregion,cell_phones_per_person
8134,usa,2017,"395,881,000.0",324459463,United States,americas,america,1.2201


Modify the `geo` column to make the geo codes uppercase instead of lowercase.

In [0]:
# Use .upper() method to capitalioze `geo` column
cell_df['geo'] = cell_df['geo'].str.upper()

In [360]:
cell_df['geo'].head()

0    AFG
1    AFG
2    AFG
3    AFG
4    AFG
Name: geo, dtype: object

## Part 3. Process data

Use the describe function, to describe your dataframe's numeric columns, and then its non-numeric columns.

(You'll see the time period ranges from 1960 to 2017, and there are 195 unique countries represented.)

In [361]:
# Excluding non-numeric columns
cell_df.describe(exclude='object')

,time,cell_phones_total,population_total,cell_phones_per_person
count,"8,590.0","8,590.0","8,590.0","8,590.0"
mean,"1,994.1934807916182","9,004,949.642905472","29,838,230.581722934",0.2796383119906869
std,14.257974607310302,"55,734,084.872179635","116,128,377.47477299",0.4542467913882122
min,"1,960.0",0.0,"4,433.0",0.0
25%,"1,983.0",0.0,"1,456,148.0",0.0
50%,"1,995.0","6,200.0","5,725,062.5",0.0016
75%,"2,006.0","1,697,652.0","18,105,812.0",0.461125
max,"2,017.0","1,474,097,000.0","1,409,517,397.0",2.4902


In [362]:
# Excluding numeric columns
cell_df.describe(exclude='number')

,geo,country,region,subregion
count,8590,8590,8590,8590
unique,195,195,4,6
top,NOR,Uzbekistan,asia,europe_central_asia
freq,46,46,2485,2324


In 2017, what were the top 5 countries with the most cell phones total?

Your list of countries should have these totals:

| country | cell phones total |
|:-------:|:-----------------:|
|    ?    |     1,474,097,000 |
|    ?    |     1,168,902,277 |
|    ?    |       458,923,202 |
|    ?    |       395,881,000 |
|    ?    |       236,488,548 |



In [0]:
# This optional code formats float numbers with comma separators
pd.options.display.float_format = '{:,}'.format

In [0]:
# Create sub dataframe that includes only the year 2017
cell_df_2017 = cell_df.loc[cell_df['time'] == 2017]

In [0]:
# Use the groupby operation that sums the cell phone yearly totals for each country
country_grouped = (cell_df_2017
            .groupby(['country'])
            .cell_phones_total.agg(['sum'])
            .rename(columns={'sum':'cell phones total'})
            .reset_index())

In [366]:
# Use the sort values function to sort by cell phones total 
country_grouped.sort_values(by=['cell phones total'], ascending=False).head(5)

,country,cell phones total
31,China,"1,474,097,000.0"
67,India,"1,168,902,277.0"
68,Indonesia,"458,923,202.0"
160,United States,"395,881,000.0"
21,Brazil,"236,488,548.0"


2017 was the first year that China had more cell phones than people.

What was the first year that the USA had more cell phones than people?

In [0]:
# Create sub dataframe that includes only rows where the country equals the United States
us_cell_df = cell_df.loc[cell_df['country'] == 'United States']

2014 was the first year that the USA had more cell phones than people!

In [368]:
# Use the .loc function to specify my output to only include rows where the cell phones total is less than the population total
us_cell_df.loc[cell_df['cell_phones_total'] > cell_df['population_total']]

,geo,time,cell_phones_total,population_total,country,region,subregion,cell_phones_per_person
8131,USA,2014,"355,500,000.0",317718779,United States,americas,america,1.1189
8132,USA,2015,"382,307,000.0",319929162,United States,americas,america,1.195
8133,USA,2016,"395,881,000.0",322179605,United States,americas,america,1.2288
8134,USA,2017,"395,881,000.0",324459463,United States,americas,america,1.2201


## Part 4. Reshape data

Create a pivot table:
- Columns: Years 2007—2017
- Rows: China, India, United States, Indonesia, Brazil (order doesn't matter)
- Values: Cell Phones Total

The table's shape should be: (5, 11)

In [0]:
# Create country names list and use as a condition to create a subset dataframe that only includes rows with the relevant countries

country_names = ['China','India','United States','Indonesia','Brazil']
condition = cell_df['country'].isin(country_names)
subset_test = cell_df[condition]

In [0]:
# Use .loc function to only include years 2007-2017
subset_final = subset.loc[cell_df['time'] > 2006]

In [371]:
# Create pivot table with required elements
country_2007_2017_pt = subset_final.pivot_table(index='country', columns='time', values='cell_phones_total')
country_2007_2017_pt

time,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
country,,,,,,,,,,,
Brazil,"120,980,103.0","150,641,403.0","169,385,584.0","196,929,978.0","234,357,507.0","248,323,703.0","271,099,799.0","280,728,796.0","257,814,274.0","244,067,356.0","236,488,548.0"
China,"547,306,000.0","641,245,000.0","747,214,000.0","859,003,000.0","986,253,000.0","1,112,155,000.0","1,229,113,000.0","1,286,093,000.0","1,291,984,200.0","1,364,934,000.0","1,474,097,000.0"
India,"233,620,000.0","346,890,000.0","525,090,000.0","752,190,000.0","893,862,478.0","864,720,917.0","886,304,245.0","944,008,677.0","1,001,056,000.0","1,127,809,000.0","1,168,902,277.0"
Indonesia,"93,386,881.0","140,578,243.0","163,676,961.0","211,290,235.0","249,805,619.0","281,963,665.0","313,226,914.0","325,582,819.0","338,948,340.0","385,573,398.0","458,923,202.0"
United States,"249,300,000.0","261,300,000.0","274,283,000.0","285,118,000.0","297,404,000.0","304,838,000.0","310,698,000.0","355,500,000.0","382,307,000.0","395,881,000.0","395,881,000.0"


In [372]:
# Check to ensure proper shape
country_2007_2017_pt.shape

(5, 11)

In [373]:
# Line graph plotting the relationship between time and total cell phones in each country
import altair as alt

alt.Chart(subset_final,height=400,width=700).mark_line().encode(
    x='time',
    y='cell_phones_total',
    color='country'
)

Chart({
  data:       geo  time  cell_phones_total  population_total        country    region  \
  1074  BRA  2007      120,980,103.0         191026637         Brazil  americas   
  1075  BRA  2008      150,641,403.0         192979029         Brazil  americas   
  1076  BRA  2009      169,385,584.0         194895996         Brazil  americas   
  1077  BRA  2010      196,929,978.0         196796269         Brazil  americas   
  1078  BRA  2011      234,357,507.0         198686688         Brazil  americas   
  1079  BRA  2012      248,323,703.0         200560983         Brazil  americas   
  1080  BRA  2013      271,099,799.0         202408632         Brazil  americas   
  1081  BRA  2014      280,728,796.0         204213133         Brazil  americas   
  1082  BRA  2015      257,814,274.0         205962108         Brazil  americas   
  1083  BRA  2016      244,067,356.0         207652865         Brazil  americas   
  1084  BRA  2017      236,488,548.0         209288278         Brazil  americas   
  1486  CHN  2007      547,306,000.0        1336800506          China      asia   
  1487  CHN  2008      641,245,000.0        1344415227          China      asia   
  1488  CHN  2009      747,214,000.0        1352068091          China      asia   
  1489  CHN  2010      859,003,000.0        1359755102          China      asia   
  1490  CHN  2011      986,253,000.0        1367480264          China      asia   
  1491  CHN  2012    1,112,155,000.0        1375198619          China      asia   
  1492  CHN  2013    1,229,113,000.0        1382793212          China      asia   
  1493  CHN  2014    1,286,093,000.0        1390110388          China      asia   
  1494  CHN  2015    1,291,984,200.0        1397028553          China      asia   
  1495  CHN  2016    1,364,934,000.0        1403500365          China      asia   
  1496  CHN  2017    1,474,097,000.0        1409517397          China      asia   
  3539  IDN  2007       93,386,881.0         232989141      Indonesia      asia   
  3540  IDN  2008      140,578,243.0         236159276      Indonesia      asia   
  3541  IDN  2009      163,676,961.0         239340478      Indonesia      asia   
  3542  IDN  2010      211,290,235.0         242524123      Indonesia      asia   
  3543  IDN  2011      249,805,619.0         245707511      Indonesia      asia   
  3544  IDN  2012      281,963,665.0         248883232      Indonesia      asia   
  3545  IDN  2013      313,226,914.0         252032263      Indonesia      asia   
  3546  IDN  2014      325,582,819.0         255131116      Indonesia      asia   
  3547  IDN  2015      338,948,340.0         258162113      Indonesia      asia   
  3548  IDN  2016      385,573,398.0         261115456      Indonesia      asia   
  3549  IDN  2017      458,923,202.0         263991379      Indonesia      asia   
  3585  IND  2007      233,620,000.0        1179681239          India      asia   
  3586  IND  2008      346,890,000.0        1197146906          India      asia   
  3587  IND  2009      525,090,000.0        1214270132          India      asia   
  3588  IND  2010      752,190,000.0        1230980691          India      asia   
  3589  IND  2011      893,862,478.0        1247236029          India      asia   
  3590  IND  2012      864,720,917.0        1263065852          India      asia   
  3591  IND  2013      886,304,245.0        1278562207          India      asia   
  3592  IND  2014      944,008,677.0        1293859294          India      asia   
  3593  IND  2015    1,001,056,000.0        1309053980          India      asia   
  3594  IND  2016    1,127,809,000.0        1324171354          India      asia   
  3595  IND  2017    1,168,902,277.0        1339180127          India      asia   
  8124  USA  2007      249,300,000.0         300595175  United States  americas   
  8125  USA  2008      261,300,000.0         303374067  United States  americas   
  8126  USA  2009      274,283,000.0         306076362  United States  americas   
  

#### OPTIONAL BONUS QUESTION!

Sort these 5 countries, by biggest increase in cell phones from 2007 to 2017.

Which country had 935,282,277 more cell phones in 2017 versus 2007?

In [0]:
# Create sub-dataframes including only the years 2007 and 2017
_2007_test = subset_test.loc[cell_df['time'] == 2007]
_2017_test = subset_test.loc[cell_df['time'] == 2017]

In [0]:
# Rename columns to avoid confusion
_2007_test =_2007_test.rename({'cell_phones_total':'2007 Cell Phone Total'},axis=1)
_2017_test =_2017_test.rename({'cell_phones_total':'2017 Cell Phone Total'},axis=1)

In [376]:
_2007_test

,geo,time,2007 Cell Phone Total,population_total,country,region,subregion,cell_phones_per_person
1074,BRA,2007,"120,980,103.0",191026637,Brazil,americas,america,0.6333
1486,CHN,2007,"547,306,000.0",1336800506,China,asia,east_asia_pacific,0.4094
3539,IDN,2007,"93,386,881.0",232989141,Indonesia,asia,east_asia_pacific,0.4008
3585,IND,2007,"233,620,000.0",1179681239,India,asia,south_asia,0.198
8124,USA,2007,"249,300,000.0",300595175,United States,americas,america,0.8294


In [0]:
# Reduce sub-dataframes to only include relevant information
_2007_test = _2007_test[['country', '2007 Cell Phone Total']]
_2017_test = _2017_test[['country', '2017 Cell Phone Total']]

In [0]:
# Merge sub-dataframes together
bonus_merge = _2007_test.merge(_2017_test)

In [379]:
# Create Cell Phone Total Increase column to see which country had the largest nominal increase in cell phones
bonus_merge['Cell Phone Total Increase'] = bonus_merge['2017 Cell Phone Total'] - bonus_merge['2007 Cell Phone Total']
bonus_merge

,country,2007 Cell Phone Total,2017 Cell Phone Total,Cell Phone Total Increase
0,Brazil,"120,980,103.0","236,488,548.0","115,508,445.0"
1,China,"547,306,000.0","1,474,097,000.0","926,791,000.0"
2,Indonesia,"93,386,881.0","458,923,202.0","365,536,321.0"
3,India,"233,620,000.0","1,168,902,277.0","935,282,277.0"
4,United States,"249,300,000.0","395,881,000.0","146,581,000.0"


**India** had the largest Nominal increase in cell phones from 2007 to 2017 with 935,282,277 answering the second question!

In [380]:
# Create Cell Phone Percentage Increase column to see which country had the largest nominal increase in cell phones
bonus_merge['Cell Phone Percentage Increase'] = ((bonus_merge['2017 Cell Phone Total'] - bonus_merge['2007 Cell Phone Total']) / bonus_merge['2007 Cell Phone Total'])
bonus_merge['Cell Phone Percentage Increase'] = bonus_merge['Cell Phone Percentage Increase'].round(decimals=4)
bonus_merge

,country,2007 Cell Phone Total,2017 Cell Phone Total,Cell Phone Total Increase,Cell Phone Percentage Increase
0,Brazil,"120,980,103.0","236,488,548.0","115,508,445.0",0.9548
1,China,"547,306,000.0","1,474,097,000.0","926,791,000.0",1.6934
2,Indonesia,"93,386,881.0","458,923,202.0","365,536,321.0",3.9142
3,India,"233,620,000.0","1,168,902,277.0","935,282,277.0",4.0034
4,United States,"249,300,000.0","395,881,000.0","146,581,000.0",0.588


**India** also had the largest percentage increase in cell phones from 2007 to 2017 with an increase of 400%!

If you have the time and curiosity, what other questions can you ask and answer with this data?

Using the features I added earlier, I am going to plot total cell phones over time for each subregion in our dataset.
- We can infer that around 2005-2010 the cell phone market for the developed market had begun to mature.
- The growth for the east pacific region exploded around the mid-2000's and is continuing to see high growth year over year.
- Around 2015 we start to see the total cell phones deecreasing in all subregions that are not in east or south asia.

In [0]:
sub_region_df = (cell_df
            .groupby(['subregion','time'])
            .cell_phones_total.agg(['sum'])
            .rename(columns={'sum':'Total Cell Phones'})
            .reset_index())

In [382]:
alt.Chart(sub_region_df,height=400,width=700).mark_line().encode(
    x='time',
    y='Total Cell Phones',
    color='subregion'
)

Chart({
  data:               subregion  time  Total Cell Phones
  0               america  1960                0.0
  1               america  1965                0.0
  2               america  1970                0.0
  3               america  1975                0.0
  4               america  1976                0.0
  5               america  1977                0.0
  6               america  1978                0.0
  7               america  1979                0.0
  8               america  1980                0.0
  9               america  1981                0.0
  10              america  1982                0.0
  11              america  1983                0.0
  12              america  1984           91,600.0
  13              america  1985          352,213.0
  14              america  1986          741,825.0
  15              america  1987        1,329,219.0
  16              america  1988        2,275,374.0
  17              america  1989        3,873,493.0
  18              america  1990        5,971,938.0
  19              america  1991        8,596,660.0
  20              america  1992       12,645,878.0
  21              america  1993       18,381,755.0
  22              america  1994       28,078,581.0
  23              america  1995       40,012,288.0
  24              america  1996       53,855,611.0
  25              america  1997       71,871,234.0
  26              america  1998       95,188,204.0
  27              america  1999      132,214,308.0
  28              america  2000      180,589,877.0
  29              america  2001      222,134,187.0
  ..                  ...   ...                ...
  246  sub_saharan_africa  1988                0.0
  247  sub_saharan_africa  1989            3,980.0
  248  sub_saharan_africa  1990            7,880.0
  249  sub_saharan_africa  1991            9,600.0
  250  sub_saharan_africa  1992           17,406.0
  251  sub_saharan_africa  1993           59,142.0
  252  sub_saharan_africa  1994          372,608.0
  253  sub_saharan_africa  1995          601,706.0
  254  sub_saharan_africa  1996        1,083,260.0
  255  sub_saharan_africa  1997        2,087,517.0
  256  sub_saharan_africa  1998        3,872,303.0
  257  sub_saharan_africa  1999        6,466,858.0
  258  sub_saharan_africa  2000       11,445,272.0
  259  sub_saharan_africa  2001       17,046,727.0
  260  sub_saharan_africa  2002       25,098,099.0
  261  sub_saharan_africa  2003       35,687,622.0
  262  sub_saharan_africa  2004       54,212,996.0
  263  sub_saharan_africa  2005       90,368,334.0
  264  sub_saharan_africa  2006      135,233,899.0
  265  sub_saharan_africa  2007      184,823,777.0
  266  sub_saharan_africa  2008      261,134,046.0
  267  sub_saharan_africa  2009      316,759,711.0
  268  sub_saharan_africa  2010      387,953,418.0
  269  sub_saharan_africa  2011      471,725,434.0
  270  sub_saharan_africa  2012      544,447,403.0
  271  sub_saharan_africa  2013      618,078,177.0
  272  sub_saharan_africa  2014      680,980,483.0
  273  sub_saharan_africa  2015      751,999,700.0
  274  sub_saharan_africa  2016      752,087,695.0
  275  sub_saharan_africa  2017      726,950,742.0
  
  [276 rows x 3 columns],
  encoding: EncodingWithFacet({
    color: Color({
      shorthand: 'subregion'
    }),
    x: X({
      shorthand: 'time'
    }),
    y: Y({
      shorthand: 'Total Cell Phones'
    })
  }),
  height: 400,
  mark: 'line',
  width: 700
})